In [1]:
!pip install /kaggle/input/kerasapplications -q
!pip install /kaggle/input/efficientnet-keras-source-code/ -q --no-deps


In [2]:
#-------------------
# importing libraries
#-------------------
import tensorflow as tf
import efficientnet.tfkeras as efn
import tensorflow_addons as tfa

import pandas as pd
import numpy as np

import os
import shutil

In [3]:
HEIGHT,WIDTH = 512,512
CHANNELS = 3
BATCH_SIZE = 32
TEST_PATH = "../input/plant-pathology-2021-fgvc8/test_images"
AUTO = tf.data.experimental.AUTOTUNE

label2id = {
    'scab': 0,
    'frog_eye_leaf_spot' : 1,
    'rust' : 2,
    'complex' : 3,
    'powdery_mildew' : 4,
}

id2label = dict([(value, key) for key, value in label2id.items()])
id2label

{0: 'scab',
 1: 'frog_eye_leaf_spot',
 2: 'rust',
 3: 'complex',
 4: 'powdery_mildew'}

In [4]:
def process_img(filepath):
    image = tf.io.read_file(filepath)
    image = tf.image.decode_jpeg(image, channels=CHANNELS)
    image = tf.image.convert_image_dtype(image, tf.float32) 
    image = tf.image.resize(image, [HEIGHT,WIDTH])
    return image


files_ls = tf.io.gfile.glob(TEST_PATH + '/*.jpg')
test_ds = tf.data.Dataset.from_tensor_slices(files_ls)
test_ds = test_ds.map(process_img,num_parallel_calls=AUTO)
test_ds = test_ds.batch(BATCH_SIZE).prefetch(AUTO)

In [5]:
MODEL_PATH = "../input/plant-pathology-2k21-baseline-tpu-training"
pred_arr = 0 
count = 0

for i,file in enumerate(os.listdir(MODEL_PATH)):
    if file.split(".")[-1] == "h5":
        count+=1
        model = tf.keras.models.load_model(os.path.join(MODEL_PATH,file))
        pred_arr += model.predict(test_ds)

pred_arr = pred_arr/count

'''model = tf.keras.models.load_model("../input/plant-pathology-2k21-baseline-tpu-training/best_model_4.h5")
pred_arr = model.predict(test_ds)'''

'model = tf.keras.models.load_model("../input/plant-pathology-2k21-baseline-tpu-training/best_model_4.h5")\npred_arr = model.predict(test_ds)'

In [6]:
THRESHOLD = 0.5
predictions = []
for i,pred in enumerate(pred_arr):
    req_pred = len(pred[pred>=THRESHOLD])
    
    if req_pred == 0:
        req_str = "healthy"
    else:
        req_args = np.argsort(-1*pred)[:req_pred]
        req_labels = [id2label[i] for i in req_args]
        req_str = " ".join(req_labels)
    predictions.append(req_str)

In [7]:
img_ls = os.listdir(TEST_PATH)
test_df = pd.DataFrame(img_ls,columns = ["image"])
test_df["labels"] = predictions
#test_df.sort_values('image',inplace=True, ascending=True)

test_df.to_csv("./submission.csv",index=False)
test_df.head()

,image,labels
0,ad8770db05586b59.jpg,frog_eye_leaf_spot scab
1,c7b03e718489f3ca.jpg,frog_eye_leaf_spot
2,85f8cb619c66b863.jpg,scab
